In [1]:
import os
from os.path import exists, join, basename, splitext

import random
import PIL
import torchvision
import cv2
import numpy as np
import torch
torch.set_grad_enabled(False)
  
import time
import matplotlib
import matplotlib.pylab as plt
plt.rcParams["axes.grid"] = False

In [2]:
model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)
model = model.eval()

C:\Users\aidan\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\aidan\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MaskRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=MaskRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [14]:
import cv2
import torch
import torchvision
import numpy as np
import random
import time
import PIL.Image

# Initialize webcam
cam = cv2.VideoCapture(0)
if not cam.isOpened(): 
    print("No camera detected!")
    exit()

# COCO labels
coco_names = [
    'unlabeled', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 
    'fire hydrant', 'street sign', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 
    'elephant', 'bear', 'zebra', 'giraffe', 'hat', 'backpack', 'umbrella', 'shoe', 'eye glasses', 'handbag', 'tie', 
    'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 
    'surfboard', 'tennis racket', 'bottle', 'plate', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 
    'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 
    'potted plant', 'bed', 'mirror', 'dining table', 'window', 'desk', 'toilet', 'door', 'tv', 'laptop', 'mouse', 
    'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'blender', 'book', 
    'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]

# Random colors for each class
colors = [[random.randint(0, 255) for _ in range(3)] for _ in coco_names]

# Load pretrained Mask R-CNN
model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)
model.eval()

while True:
    ret, frame = cam.read()
    if not ret:
        print("Failed to capture image")
        break

    # Convert frame to PIL Image and Tensor
    image = PIL.Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    image_tensor = torchvision.transforms.functional.to_tensor(image).unsqueeze(0)

    # Run inference
    t = time.time()
    with torch.no_grad():
        output = model(image_tensor)[0]
    print(f'Executed in {time.time() - t:.3f}s')

    # Copy frame for drawing
    result_image = frame.copy()
    mask_overlay = np.zeros_like(frame, dtype=np.uint8)  # Create a blank mask

    color = random.choice(colors)

    # Draw bounding boxes, labels, and masks
    for box, label, score, mask in zip(output['boxes'], output['labels'], output['scores'], output['masks']):
        if score > 0.5:  # Confidence threshold
            x1, y1, x2, y2 = map(int, box.tolist())

            # Draw bounding box
            thickness = 2
            cv2.rectangle(result_image, (x1, y1), (x2, y2), color, thickness)

            # Draw label
            text = f"{coco_names[label]}: {score:.1f}%"
            font_scale = 0.5
            font_thickness = 1
            text_size, _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, font_scale, font_thickness)
            text_x, text_y = x1, y1 - 5
            cv2.rectangle(result_image, (text_x, text_y - text_size[1]), (text_x + text_size[0], text_y), color, -1)
            cv2.putText(result_image, text, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (255, 255, 255), font_thickness)

            # Process the mask
            mask = mask.squeeze().cpu().numpy()  # Convert to numpy array
            mask = (mask > 0.5).astype(np.uint8)  # Threshold the mask
            mask_resized = cv2.resize(mask, (x2 - x1, y2 - y1))  # Resize mask
            mask_resized = mask_resized[:, :, np.newaxis]  # Add channel dimension

            # Color the mask
            mask_color = np.array(color, dtype=np.uint8)    
            mask_colored = mask_resized * mask_color  # Apply color

            # Overlay mask onto the blank overlay image
            mask_overlay[y1:y2, x1:x2] = np.maximum(mask_overlay[y1:y2, x1:x2], mask_colored)

    # Blend mask overlay with the result image
    alpha = 0.5  # Transparency factor
    result_image = cv2.addWeighted(result_image, 1, mask_overlay, alpha, 0)

    # Show result
    cv2.imshow("Mask R-CNN - Real Time", result_image)

    # Exit on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Cleanup
cam.release()
cv2.destroyAllWindows()


Executed in 1.696s
Executed in 1.777s
Executed in 1.666s
Executed in 1.708s
Executed in 1.793s
Executed in 1.755s
Executed in 1.744s
Executed in 1.809s
Executed in 1.778s
Executed in 1.805s
Executed in 1.857s
Executed in 1.854s
Executed in 1.807s
Executed in 1.744s
Executed in 1.748s
Executed in 1.718s
Executed in 1.779s
Executed in 1.859s
Executed in 1.744s
